In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score,classification_report,f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 

import math
import torch
from torch import nn
from torch import optim
from torchvision.datasets import MNIST
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.utils.data as utils
from torch.optim import Adam 

In [4]:
transform = transforms.Compose([
transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

In [8]:
trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

In [21]:
trainloader = torch.utils.data.DataLoader(
        datasets.MNIST('MNIST_data/', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('MNIST_data/', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=16, shuffle=True)

In [11]:
len(trainloader)

7500

In [12]:
len(train_loader)

7500

In [13]:
len(test_loader)

1250

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
device = torch.device('cpu')

cuda


In [15]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1' , nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2' , nn.ReLU())
model.add_module('l3', nn.Linear(200, 10))

model.to(device)

Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=10, bias=True)
)

In [86]:
a = list(model.l1.weight)

In [87]:
len(a)

200

In [89]:
len(a[0])

784

In [10]:
a[0][0]

tensor(0.0431, grad_fn=<SelectBackward>)

In [51]:
p = torch.eye(200)

In [52]:
p

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [53]:
p = p*10

In [56]:
 p

tensor([[10.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0., 10.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0., 10.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ..., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0., 10.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0., 10.]])

In [57]:
b = 10*torch.eye(784)

In [59]:
m_200 = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(200), p)
m_784 = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(784), b)

In [61]:
def call_pr_loss(a , m ):
    s = 0 
    c = 0
    for k in a:

        s -= m.log_prob(k)
        c += 1
    return s/c

In [62]:
call_pr_loss(list(model.l2.weight) , m_200 )

tensor(414.0628, grad_fn=<DivBackward0>)

In [63]:
call_pr_loss(list(model.l1.weight) , m_784 )

tensor(1623.0780, grad_fn=<DivBackward0>)

In [18]:
m_200 = 1
m_784 = 1

In [69]:
def call_pr_loss(a , variance):
    var = variance**2
    log = math.log()
    return torch.sum(a**2) / variance**2 / 2

In [16]:
def call_pr_loss(a , variance):
    k = len(a[0])
    l = torch.sum(a**2)/ variance**2 /2/k
    return k/2*math.log(2*math.pi) + 0.5*math.log(variance**k) + l

In [67]:
call_pr_loss(model.l1.weight , 10)

tensor(1623.0615, grad_fn=<AddBackward0>)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 3):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        
        
        
        loss = loss_0 + loss_1+ loss_2+ loss_3
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()

Train Epoch: 1 [15984/10000 (27%)]	Loss: 1088.200073 
Train Epoch: 1 [31984/10000 (53%)]	Loss: 1088.386719 
Train Epoch: 1 [47984/10000 (80%)]	Loss: 1088.374268 
Train Epoch: 2 [3984/10000 (7%)]	Loss: 1088.188599 
Train Epoch: 2 [19984/10000 (33%)]	Loss: 1088.276245 
Train Epoch: 2 [35984/10000 (60%)]	Loss: 1088.148804 
Train Epoch: 2 [51984/10000 (87%)]	Loss: 1088.139404 


In [23]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        output = model(data)
        test_loss += criterion(output, target).item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0051, Accuracy: 9757/10000 (98%)



In [24]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)


Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 3):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        
        loss = loss_0 + loss_1+ loss_2+ loss_3 + loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 100  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()

Train Epoch: 1 [1584/60000 (3%)]	Loss: 1272.964966 
Train Epoch: 1 [3184/60000 (5%)]	Loss: 1272.647583 
Train Epoch: 1 [4784/60000 (8%)]	Loss: 1272.440552 
Train Epoch: 1 [6384/60000 (11%)]	Loss: 1272.543213 
Train Epoch: 1 [7984/60000 (13%)]	Loss: 1272.455566 
Train Epoch: 1 [9584/60000 (16%)]	Loss: 1272.529663 
Train Epoch: 1 [11184/60000 (19%)]	Loss: 1272.099243 
Train Epoch: 1 [12784/60000 (21%)]	Loss: 1272.302246 
Train Epoch: 1 [14384/60000 (24%)]	Loss: 1272.461670 
Train Epoch: 1 [15984/60000 (27%)]	Loss: 1272.166748 
Train Epoch: 1 [17584/60000 (29%)]	Loss: 1272.083496 
Train Epoch: 1 [19184/60000 (32%)]	Loss: 1272.129883 
Train Epoch: 1 [20784/60000 (35%)]	Loss: 1272.066895 
Train Epoch: 1 [22384/60000 (37%)]	Loss: 1272.062866 
Train Epoch: 1 [23984/60000 (40%)]	Loss: 1272.041626 
Train Epoch: 1 [25584/60000 (43%)]	Loss: 1272.024170 
Train Epoch: 1 [27184/60000 (45%)]	Loss: 1272.112671 
Train Epoch: 1 [28784/60000 (48%)]	Loss: 1272.034912 
Train Epoch: 1 [30384/60000 (51%)]	Lo

In [12]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        output = model(data)
        test_loss += criterion(output, target).item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0333, Accuracy: 57630/60000 (96%)



In [13]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)



Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 3):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = loss_0 + loss_1+ loss_2+ loss_3 + loss_4 + loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 100  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()

Train Epoch: 1 [396/60000 (1%)]	Loss: 1457.840210 
Train Epoch: 1 [796/60000 (1%)]	Loss: 1457.165283 
Train Epoch: 1 [1196/60000 (2%)]	Loss: 1456.364136 
Train Epoch: 1 [1596/60000 (3%)]	Loss: 1457.063232 
Train Epoch: 1 [1996/60000 (3%)]	Loss: 1456.251831 
Train Epoch: 1 [2396/60000 (4%)]	Loss: 1456.188843 
Train Epoch: 1 [2796/60000 (5%)]	Loss: 1456.549438 
Train Epoch: 1 [3196/60000 (5%)]	Loss: 1456.304199 
Train Epoch: 1 [3596/60000 (6%)]	Loss: 1455.958008 
Train Epoch: 1 [3996/60000 (7%)]	Loss: 1456.538574 
Train Epoch: 1 [4396/60000 (7%)]	Loss: 1457.051758 
Train Epoch: 1 [4796/60000 (8%)]	Loss: 1455.826050 
Train Epoch: 1 [5196/60000 (9%)]	Loss: 1455.834473 
Train Epoch: 1 [5596/60000 (9%)]	Loss: 1456.959595 
Train Epoch: 1 [5996/60000 (10%)]	Loss: 1456.251465 
Train Epoch: 1 [6396/60000 (11%)]	Loss: 1457.694336 
Train Epoch: 1 [6796/60000 (11%)]	Loss: 1456.366211 
Train Epoch: 1 [7196/60000 (12%)]	Loss: 1456.144653 
Train Epoch: 1 [7596/60000 (13%)]	Loss: 1455.980225 
Train Epo

In [15]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        output = model(data)
        test_loss += criterion(output, target).item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0360, Accuracy: 57473/60000 (96%)



In [16]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)



Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 3):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = loss_0 + loss_1+ loss_2+ loss_3 + loss_4 + loss_f + loss_5
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 100  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()

Train Epoch: 1 [396/60000 (1%)]	Loss: 1641.847046 
Train Epoch: 1 [796/60000 (1%)]	Loss: 1641.715332 
Train Epoch: 1 [1196/60000 (2%)]	Loss: 1640.577759 
Train Epoch: 1 [1596/60000 (3%)]	Loss: 1640.987549 
Train Epoch: 1 [1996/60000 (3%)]	Loss: 1640.054810 
Train Epoch: 1 [2396/60000 (4%)]	Loss: 1641.414307 
Train Epoch: 1 [2796/60000 (5%)]	Loss: 1640.253906 
Train Epoch: 1 [3196/60000 (5%)]	Loss: 1640.023682 
Train Epoch: 1 [3596/60000 (6%)]	Loss: 1640.799805 
Train Epoch: 1 [3996/60000 (7%)]	Loss: 1640.443848 
Train Epoch: 1 [4396/60000 (7%)]	Loss: 1640.130493 
Train Epoch: 1 [4796/60000 (8%)]	Loss: 1640.552490 
Train Epoch: 1 [5196/60000 (9%)]	Loss: 1640.397949 
Train Epoch: 1 [5596/60000 (9%)]	Loss: 1640.360718 
Train Epoch: 1 [5996/60000 (10%)]	Loss: 1640.708740 
Train Epoch: 1 [6396/60000 (11%)]	Loss: 1639.787598 
Train Epoch: 1 [6796/60000 (11%)]	Loss: 1639.922241 
Train Epoch: 1 [7196/60000 (12%)]	Loss: 1640.192383 
Train Epoch: 1 [7596/60000 (13%)]	Loss: 1639.908325 
Train Epo

In [26]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        output = model(data)
        test_loss += criterion(output, target).item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0055, Accuracy: 9732/10000 (97%)



In [27]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)



Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 3):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = loss_0 + loss_1+ loss_2+ loss_3 + loss_4 + loss_f + loss_5 + loss_6
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 100  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()

Train Epoch: 1 [1584/60000 (3%)]	Loss: 1824.016357 
Train Epoch: 1 [3184/60000 (5%)]	Loss: 1824.472046 
Train Epoch: 1 [4784/60000 (8%)]	Loss: 1824.095215 
Train Epoch: 1 [6384/60000 (11%)]	Loss: 1823.915527 
Train Epoch: 1 [7984/60000 (13%)]	Loss: 1824.086182 
Train Epoch: 1 [9584/60000 (16%)]	Loss: 1823.825439 
Train Epoch: 1 [11184/60000 (19%)]	Loss: 1823.993652 
Train Epoch: 1 [12784/60000 (21%)]	Loss: 1823.994507 
Train Epoch: 1 [14384/60000 (24%)]	Loss: 1823.631714 
Train Epoch: 1 [15984/60000 (27%)]	Loss: 1824.170898 
Train Epoch: 1 [17584/60000 (29%)]	Loss: 1823.708984 
Train Epoch: 1 [19184/60000 (32%)]	Loss: 1823.589478 
Train Epoch: 1 [20784/60000 (35%)]	Loss: 1823.599731 
Train Epoch: 1 [22384/60000 (37%)]	Loss: 1823.715820 
Train Epoch: 1 [23984/60000 (40%)]	Loss: 1823.521851 
Train Epoch: 1 [25584/60000 (43%)]	Loss: 1823.559326 
Train Epoch: 1 [27184/60000 (45%)]	Loss: 1823.699707 
Train Epoch: 1 [28784/60000 (48%)]	Loss: 1823.621704 
Train Epoch: 1 [30384/60000 (51%)]	Lo

In [29]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        output = model(data)
        test_loss += criterion(output, target).item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0083, Accuracy: 9612/10000 (96%)



In [30]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('l7', nn.Linear(200, 200))
model.add_module('r7', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)



Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (l7): Linear(in_features=200, out_features=200, bias=True)
  (r7): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 3):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_7 = call_pr_loss(model.l7.weight , m_200)
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = loss_0 + loss_1+ loss_2+ loss_3 + loss_4 + loss_f + loss_5 + loss_6 + loss_7
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 100  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()

Train Epoch: 1 [1584/60000 (3%)]	Loss: 2008.618896 
Train Epoch: 1 [3184/60000 (5%)]	Loss: 2008.880249 
Train Epoch: 1 [4784/60000 (8%)]	Loss: 2008.625244 
Train Epoch: 1 [6384/60000 (11%)]	Loss: 2008.532715 
Train Epoch: 1 [7984/60000 (13%)]	Loss: 2008.613525 
Train Epoch: 1 [9584/60000 (16%)]	Loss: 2008.244385 
Train Epoch: 1 [11184/60000 (19%)]	Loss: 2007.592773 
Train Epoch: 1 [12784/60000 (21%)]	Loss: 2008.119995 
Train Epoch: 1 [14384/60000 (24%)]	Loss: 2007.307495 
Train Epoch: 1 [15984/60000 (27%)]	Loss: 2007.670044 
Train Epoch: 1 [17584/60000 (29%)]	Loss: 2007.780029 
Train Epoch: 1 [19184/60000 (32%)]	Loss: 2007.625610 
Train Epoch: 1 [20784/60000 (35%)]	Loss: 2007.404907 
Train Epoch: 1 [22384/60000 (37%)]	Loss: 2007.579224 
Train Epoch: 1 [23984/60000 (40%)]	Loss: 2007.571655 
Train Epoch: 1 [25584/60000 (43%)]	Loss: 2007.388916 
Train Epoch: 1 [27184/60000 (45%)]	Loss: 2007.348999 
Train Epoch: 1 [28784/60000 (48%)]	Loss: 2007.244629 
Train Epoch: 1 [30384/60000 (51%)]	Lo

In [32]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        output = model(data)
        test_loss += criterion(output, target).item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0094, Accuracy: 9585/10000 (96%)



# лосс на final не учитываем

In [98]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('l7', nn.Linear(200, 200))
model.add_module('r7', nn.ReLU())
model.add_module('l8', nn.Linear(200, 200))
model.add_module('r8', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)



Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (l7): Linear(in_features=200, out_features=200, bias=True)
  (r7): ReLU()
  (l8): Linear(in_features=200, out_features=200, bias=True)
  (r8): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [99]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 10):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_7 = call_pr_loss(model.l7.weight , m_200)
        loss_8 = call_pr_loss(model.l8.weight , m_200)
        
        #loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = loss_0   + loss_1+ loss_2+ loss_3 + loss_4  + loss_5 + loss_6 + loss_7 + loss_8 #+loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()
    eva(model , trainloader)

Train Epoch: 1 [3996/60000 (7%)]	Loss: 2009.298950 
Train Epoch: 1 [7996/60000 (13%)]	Loss: 2009.256836 
Train Epoch: 1 [11996/60000 (20%)]	Loss: 2009.238403 
Train Epoch: 1 [15996/60000 (27%)]	Loss: 2009.255859 
Train Epoch: 1 [19996/60000 (33%)]	Loss: 2009.236938 
Train Epoch: 1 [23996/60000 (40%)]	Loss: 2009.258423 
Train Epoch: 1 [27996/60000 (47%)]	Loss: 2009.303711 
Train Epoch: 1 [31996/60000 (53%)]	Loss: 2009.276001 
Train Epoch: 1 [35996/60000 (60%)]	Loss: 2009.264771 
Train Epoch: 1 [39996/60000 (67%)]	Loss: 2009.228394 
Train Epoch: 1 [43996/60000 (73%)]	Loss: 2009.250610 
Train Epoch: 1 [47996/60000 (80%)]	Loss: 2009.242065 
Train Epoch: 1 [51996/60000 (87%)]	Loss: 2009.237183 
Train Epoch: 1 [55996/60000 (93%)]	Loss: 2009.244751 
Train Epoch: 1 [59996/60000 (100%)]	Loss: 2009.249878 

Test set: Average loss: 0.5754, Accuracy: 6742/60000 (11%)

Train Epoch: 2 [3996/60000 (7%)]	Loss: 2009.219604 
Train Epoch: 2 [7996/60000 (13%)]	Loss: 2009.290771 
Train Epoch: 2 [11996/6000

In [44]:

model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        output = model(data)
        test_loss += criterion(output, target).item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(trainloader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(trainloader.dataset),
    100. * correct / len(trainloader.dataset)))


Test set: Average loss: 0.5753, Accuracy: 6742/60000 (11%)



# умножили на 1000

In [46]:
def eva(model , trainloader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in trainloader:
            data, target = data.to(device), target.to(device)
            data = data.view(data.size(0), -1)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(trainloader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(trainloader.dataset),
        100. * correct / len(trainloader.dataset)))

In [100]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('l7', nn.Linear(200, 200))
model.add_module('r7', nn.ReLU())
model.add_module('l8', nn.Linear(200, 200))
model.add_module('r8', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)


Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (l7): Linear(in_features=200, out_features=200, bias=True)
  (r7): ReLU()
  (l8): Linear(in_features=200, out_features=200, bias=True)
  (r8): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [101]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 10):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_7 = call_pr_loss(model.l7.weight , m_200)
        loss_8 = call_pr_loss(model.l8.weight , m_200)
        
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = 1000*loss_0   + loss_1+ loss_2+ loss_3 + loss_4  + loss_5 + loss_6 + loss_7 + loss_8 +loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()
    eva(model , trainloader)

Train Epoch: 1 [3996/60000 (7%)]	Loss: 3428.832520 
Train Epoch: 1 [7996/60000 (13%)]	Loss: 3454.763672 
Train Epoch: 1 [11996/60000 (20%)]	Loss: 2546.075684 
Train Epoch: 1 [15996/60000 (27%)]	Loss: 2988.069336 
Train Epoch: 1 [19996/60000 (33%)]	Loss: 2978.840332 
Train Epoch: 1 [23996/60000 (40%)]	Loss: 2531.612793 
Train Epoch: 1 [27996/60000 (47%)]	Loss: 2437.631836 
Train Epoch: 1 [31996/60000 (53%)]	Loss: 2292.927734 
Train Epoch: 1 [35996/60000 (60%)]	Loss: 2464.980957 
Train Epoch: 1 [39996/60000 (67%)]	Loss: 2201.720703 
Train Epoch: 1 [43996/60000 (73%)]	Loss: 2521.985107 
Train Epoch: 1 [47996/60000 (80%)]	Loss: 2405.917725 
Train Epoch: 1 [51996/60000 (87%)]	Loss: 2758.216064 
Train Epoch: 1 [55996/60000 (93%)]	Loss: 3505.036865 
Train Epoch: 1 [59996/60000 (100%)]	Loss: 2218.177246 

Test set: Average loss: 0.0815, Accuracy: 54882/60000 (91%)

Train Epoch: 2 [3996/60000 (7%)]	Loss: 3044.880615 
Train Epoch: 2 [7996/60000 (13%)]	Loss: 2328.531494 
Train Epoch: 2 [11996/600

In [50]:
eva(model , trainloader)


Test set: Average loss: 0.0764, Accuracy: 54479/60000 (91%)



# увеличили дисперсию до 10

In [71]:
m_200 = 10
m_784 = 10

In [91]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('l7', nn.Linear(200, 200))
model.add_module('r7', nn.ReLU())
model.add_module('l8', nn.Linear(200, 200))
model.add_module('r8', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)

Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (l7): Linear(in_features=200, out_features=200, bias=True)
  (r7): ReLU()
  (l8): Linear(in_features=200, out_features=200, bias=True)
  (r8): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [92]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 10):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_7 = call_pr_loss(model.l7.weight , m_200)
        loss_8 = call_pr_loss(model.l8.weight , m_200)
        
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = loss_0   + loss_1+ loss_2+ loss_3 + loss_4  + loss_5 + loss_6 + loss_7 + loss_8 +loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()
    eva(model , trainloader)

Train Epoch: 1 [3996/60000 (7%)]	Loss: 4936.206543 
Train Epoch: 1 [7996/60000 (13%)]	Loss: 4936.980469 
Train Epoch: 1 [11996/60000 (20%)]	Loss: 4936.188477 
Train Epoch: 1 [15996/60000 (27%)]	Loss: 4936.123047 
Train Epoch: 1 [19996/60000 (33%)]	Loss: 4936.562012 
Train Epoch: 1 [23996/60000 (40%)]	Loss: 4936.707520 
Train Epoch: 1 [27996/60000 (47%)]	Loss: 4935.689941 
Train Epoch: 1 [31996/60000 (53%)]	Loss: 4936.292969 
Train Epoch: 1 [35996/60000 (60%)]	Loss: 4935.620605 
Train Epoch: 1 [39996/60000 (67%)]	Loss: 4936.000488 
Train Epoch: 1 [43996/60000 (73%)]	Loss: 4935.735352 
Train Epoch: 1 [47996/60000 (80%)]	Loss: 4936.836914 
Train Epoch: 1 [51996/60000 (87%)]	Loss: 4937.213379 
Train Epoch: 1 [55996/60000 (93%)]	Loss: 4935.500977 
Train Epoch: 1 [59996/60000 (100%)]	Loss: 4935.719238 

Test set: Average loss: 0.0965, Accuracy: 53898/60000 (90%)

Train Epoch: 2 [3996/60000 (7%)]	Loss: 4935.476562 
Train Epoch: 2 [7996/60000 (13%)]	Loss: 4935.507324 
Train Epoch: 2 [11996/600

In [74]:
eva(model , trainloader)


Test set: Average loss: 0.0631, Accuracy: 55614/60000 (93%)



# умножили на 1000 и дисперсия 10

In [93]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('l7', nn.Linear(200, 200))
model.add_module('r7', nn.ReLU())
model.add_module('l8', nn.Linear(200, 200))
model.add_module('r8', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)

Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (l7): Linear(in_features=200, out_features=200, bias=True)
  (r7): ReLU()
  (l8): Linear(in_features=200, out_features=200, bias=True)
  (r8): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [94]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 10):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_7 = call_pr_loss(model.l7.weight , m_200)
        loss_8 = call_pr_loss(model.l8.weight , m_200)
        
        loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = 1000*loss_0   + loss_1+ loss_2+ loss_3 + loss_4  + loss_5 + loss_6 + loss_7 + loss_8 +loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()
    eva(model , trainloader)

Train Epoch: 1 [3996/60000 (7%)]	Loss: 7267.065918 
Train Epoch: 1 [7996/60000 (13%)]	Loss: 7238.328613 
Train Epoch: 1 [11996/60000 (20%)]	Loss: 7255.943848 
Train Epoch: 1 [15996/60000 (27%)]	Loss: 7242.356445 
Train Epoch: 1 [19996/60000 (33%)]	Loss: 7243.780762 
Train Epoch: 1 [23996/60000 (40%)]	Loss: 7216.163574 
Train Epoch: 1 [27996/60000 (47%)]	Loss: 7234.319336 
Train Epoch: 1 [31996/60000 (53%)]	Loss: 7260.495117 
Train Epoch: 1 [35996/60000 (60%)]	Loss: 7272.459473 
Train Epoch: 1 [39996/60000 (67%)]	Loss: 7225.554199 
Train Epoch: 1 [43996/60000 (73%)]	Loss: 7221.977051 
Train Epoch: 1 [47996/60000 (80%)]	Loss: 7271.125977 
Train Epoch: 1 [51996/60000 (87%)]	Loss: 7232.774902 
Train Epoch: 1 [55996/60000 (93%)]	Loss: 7233.566895 
Train Epoch: 1 [59996/60000 (100%)]	Loss: 7259.659668 

Test set: Average loss: 0.5758, Accuracy: 5923/60000 (10%)

Train Epoch: 2 [3996/60000 (7%)]	Loss: 7257.117188 
Train Epoch: 2 [7996/60000 (13%)]	Loss: 7227.924316 
Train Epoch: 2 [11996/6000

In [77]:
eva(model , trainloader)


Test set: Average loss: 0.5760, Accuracy: 5949/60000 (10%)



# все тоже самое но без последнего лосса

In [84]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('l7', nn.Linear(200, 200))
model.add_module('r7', nn.ReLU())
model.add_module('l8', nn.Linear(200, 200))
model.add_module('r8', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)

Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (l7): Linear(in_features=200, out_features=200, bias=True)
  (r7): ReLU()
  (l8): Linear(in_features=200, out_features=200, bias=True)
  (r8): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [86]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 10):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_7 = call_pr_loss(model.l7.weight , m_200)
        loss_8 = call_pr_loss(model.l8.weight , m_200)
        
        #loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = loss_0   + loss_1+ loss_2+ loss_3 + loss_4  + loss_5 + loss_6 + loss_7 + loss_8 #+loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()
    eva(model , trainloader)

Train Epoch: 1 [3996/60000 (7%)]	Loss: 4521.671387 
Train Epoch: 1 [7996/60000 (13%)]	Loss: 4521.910645 
Train Epoch: 1 [11996/60000 (20%)]	Loss: 4521.453613 
Train Epoch: 1 [15996/60000 (27%)]	Loss: 4522.364258 
Train Epoch: 1 [19996/60000 (33%)]	Loss: 4521.476562 
Train Epoch: 1 [23996/60000 (40%)]	Loss: 4521.411133 
Train Epoch: 1 [27996/60000 (47%)]	Loss: 4521.409668 
Train Epoch: 1 [31996/60000 (53%)]	Loss: 4521.402344 
Train Epoch: 1 [35996/60000 (60%)]	Loss: 4521.419922 
Train Epoch: 1 [39996/60000 (67%)]	Loss: 4521.751953 
Train Epoch: 1 [43996/60000 (73%)]	Loss: 4521.426758 
Train Epoch: 1 [47996/60000 (80%)]	Loss: 4521.448730 
Train Epoch: 1 [51996/60000 (87%)]	Loss: 4521.501953 
Train Epoch: 1 [55996/60000 (93%)]	Loss: 4522.091309 
Train Epoch: 1 [59996/60000 (100%)]	Loss: 4522.105957 

Test set: Average loss: 0.0621, Accuracy: 56138/60000 (94%)

Train Epoch: 2 [3996/60000 (7%)]	Loss: 4522.489258 
Train Epoch: 2 [7996/60000 (13%)]	Loss: 4521.407227 
Train Epoch: 2 [11996/600

In [ ]:
eva(model , trainloader)

In [95]:
model = nn.Sequential()
model.add_module('l1', nn.Linear(784, 200))
model.add_module('r1', nn.ReLU())
model.add_module('l2', nn.Linear(200, 200))
model.add_module('r2', nn.ReLU())
model.add_module('l3', nn.Linear(200, 200))
model.add_module('r3', nn.ReLU())
model.add_module('l4', nn.Linear(200, 200))
model.add_module('r4', nn.ReLU())
model.add_module('l5', nn.Linear(200, 200))
model.add_module('r5', nn.ReLU())
model.add_module('l6', nn.Linear(200, 200))
model.add_module('r6', nn.ReLU())
model.add_module('l7', nn.Linear(200, 200))
model.add_module('r7', nn.ReLU())
model.add_module('l8', nn.Linear(200, 200))
model.add_module('r8', nn.ReLU())
model.add_module('lf', nn.Linear(200, 10))

model.to(device)

Sequential(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=200, out_features=200, bias=True)
  (r3): ReLU()
  (l4): Linear(in_features=200, out_features=200, bias=True)
  (r4): ReLU()
  (l5): Linear(in_features=200, out_features=200, bias=True)
  (r5): ReLU()
  (l6): Linear(in_features=200, out_features=200, bias=True)
  (r6): ReLU()
  (l7): Linear(in_features=200, out_features=200, bias=True)
  (r7): ReLU()
  (l8): Linear(in_features=200, out_features=200, bias=True)
  (r8): ReLU()
  (lf): Linear(in_features=200, out_features=10, bias=True)
)

In [96]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

model.train()
step = 0 
loss_history = []
for epoch in range(1, 10):
    for batch_idx, (data, target) in enumerate(trainloader):
        
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss_0 = criterion(output, target)
        
        loss_1 = call_pr_loss(model.l1.weight , m_784)
        
        loss_2 = call_pr_loss(model.l2.weight , m_200)
        loss_3 = call_pr_loss(model.l3.weight , m_200)
        loss_4 = call_pr_loss(model.l4.weight , m_200)
        loss_5 = call_pr_loss(model.l5.weight , m_200)
        loss_6 = call_pr_loss(model.l6.weight , m_200)
        loss_7 = call_pr_loss(model.l7.weight , m_200)
        loss_8 = call_pr_loss(model.l8.weight , m_200)
        
        #loss_f = call_pr_loss(model.lf.weight , m_200)
        
        
        loss = 1000*loss_0   + loss_1+ loss_2+ loss_3 + loss_4  + loss_5 + loss_6 + loss_7 + loss_8 #+loss_f
        loss.backward()
        optimizer.step()
        step += 1
        loss_history.append(loss.item())
        if step % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} '.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))
    scheduler.step()
    eva(model , trainloader)

Train Epoch: 1 [3996/60000 (7%)]	Loss: 6569.334961 
Train Epoch: 1 [7996/60000 (13%)]	Loss: 6374.471680 
Train Epoch: 1 [11996/60000 (20%)]	Loss: 5559.868164 
Train Epoch: 1 [15996/60000 (27%)]	Loss: 5311.867188 
Train Epoch: 1 [19996/60000 (33%)]	Loss: 4954.538086 
Train Epoch: 1 [23996/60000 (40%)]	Loss: 4890.626465 
Train Epoch: 1 [27996/60000 (47%)]	Loss: 4630.379883 
Train Epoch: 1 [31996/60000 (53%)]	Loss: 4782.982422 
Train Epoch: 1 [35996/60000 (60%)]	Loss: 5086.526855 
Train Epoch: 1 [39996/60000 (67%)]	Loss: 4820.097168 
Train Epoch: 1 [43996/60000 (73%)]	Loss: 5841.403320 
Train Epoch: 1 [47996/60000 (80%)]	Loss: 4611.826660 
Train Epoch: 1 [51996/60000 (87%)]	Loss: 5313.235352 
Train Epoch: 1 [55996/60000 (93%)]	Loss: 4717.516602 
Train Epoch: 1 [59996/60000 (100%)]	Loss: 5568.170898 

Test set: Average loss: 0.0817, Accuracy: 54989/60000 (92%)

Train Epoch: 2 [3996/60000 (7%)]	Loss: 5683.408203 
Train Epoch: 2 [7996/60000 (13%)]	Loss: 4641.473145 
Train Epoch: 2 [11996/600

In [83]:
eva(model , trainloader)


Test set: Average loss: 0.0656, Accuracy: 55585/60000 (93%)

